In [1]:
def fetch_drivers_data(conn):
    query = '''
    SELECT
        driverId,
        forename,
        surname,
        dob,
        nationality
    FROM
        drivers
    '''
    return pd.read_sql_query(query, conn)

In [3]:
import json
import sqlite3

def create_driver_years_json(conn, output_file_path):
    query = '''
    SELECT
        lt.driverId,
        strftime('%Y', r.date) AS year
    FROM
        lap_times lt
    JOIN
        races r ON lt.raceId = r.raceId
    GROUP BY
        lt.driverId, year
    '''

    cursor = conn.cursor()
    cursor.execute(query)

    driver_years_data = {}

    for row in cursor.fetchall():
        driver_id, year = row
        driver_id_str = str(driver_id)

        if driver_id_str not in driver_years_data:
            driver_years_data[driver_id_str] = []

        driver_years_data[driver_id_str].append(int(year))

    # Write the data to a JSON file
    with open(output_file_path, 'w') as json_file:
        json.dump(driver_years_data, json_file, indent=2)

# Connect to the SQLite database
conn = sqlite3.connect('formula1-originalDB.db')

# Specify the path for the output JSON file
output_json_path = 'driver_years.json'

# Create the JSON file
create_driver_years_json(conn, output_json_path)

conn.close()


In [6]:
import sqlite3
import pandas as pd

def create_driver_years_df(conn):
    query = '''
    SELECT
        lt.driverId,
        strftime('%Y', r.date) AS year
    FROM
        lap_times lt
    JOIN
        races r ON lt.raceId = r.raceId
    GROUP BY
        lt.driverId, year
    '''

    cursor = conn.cursor()
    cursor.execute(query)

    driver_years_data = []

    for row in cursor.fetchall():
        driver_id, year = row
        driver_years_data.append({'driverId': driver_id, 'year': int(year)})

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(driver_years_data)

    return df

# Connect to the SQLite database
conn = sqlite3.connect('formula1-originalDB.db')

# Create the DataFrame
driver_years_df = create_driver_years_df(conn)

print(driver_years_df)

conn.close()


     driverId  year
0           1  2007
1           1  2008
2           1  2009
3           1  2010
4           1  2011
..        ...   ...
663       855  2023
664       856  2022
665       856  2023
666       857  2023
667       858  2023

[668 rows x 2 columns]


In [7]:
import sqlite3
import pandas as pd

def create_driver_years_df(conn):
    query = '''
    SELECT
        lt.driverId,
        (strftime('%Y', MAX(r.date)) - strftime('%Y', MIN(r.date)) + 1) AS years_competed
    FROM
        lap_times lt
    JOIN
        races r ON lt.raceId = r.raceId
    GROUP BY
        lt.driverId
    '''

    cursor = conn.cursor()
    cursor.execute(query)

    driver_years_data = []

    for row in cursor.fetchall():
        driver_id, years_competed = row
        driver_years_data.append({'driverId': driver_id, 'years_competed': years_competed})

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(driver_years_data)

    return df

# Connect to the SQLite database
conn = sqlite3.connect('formula1-originalDB.db')

# Create the DataFrame
driver_years_df = create_driver_years_df(conn)

# Save the DataFrame as a JSON file
output_json_path = 'driver_years.json'
driver_years_df.to_json(output_json_path, orient='records', indent=2)

conn.close()


In [17]:
all1 = driver_years_df[driver_years_df['years_competed'] >= 1].shape[0]

one = driver_years_df[driver_years_df['years_competed'] == 1].shape[0]
two = driver_years_df[driver_years_df['years_competed'] == 2].shape[0]
three = driver_years_df[driver_years_df['years_competed'] == 3].shape[0]
four = driver_years_df[driver_years_df['years_competed'] == 4].shape[0]
five = driver_years_df[driver_years_df['years_competed'] == 5].shape[0]
more_than_six_years = driver_years_df[driver_years_df['years_competed'] >=6].shape[0]

print(f"All: {all1}")

print(f"One: {one}")
print(f"Two: {two}")
print(f"Three: {three}")
print(f"Four: {four}")
print(f"Five: {five}")
print(f"Number of drivers with more than 6 years: {more_than_six_years}")

All: 139
One: 33
Two: 29
Three: 12
Four: 2
Five: 9
Number of drivers with more than 6 years: 54
